In [17]:
import pandas as pd
import numpy as np
import openrouteservice as ors
import folium

In [2]:
properties = pd.read_csv("../data/curated/distance_to_property.csv")

In [3]:
property_position = properties[['prop_lat', 'prop_long']]
school_position = properties[['school_lat', 'school_long']]
hospital_position = properties[['hospital_lat', 'hospital_long']]
station_position = properties[['station_lat', 'station_long']]
facility_position = properties[['facility_lat', 'facility_long']]

In [5]:
client = ors.Client(key='5b3ce3597851110001cf62487c93bb74ac4048998460ee4d5ed60ec2')

In [10]:
test = pd.merge(property_position.reset_index(), school_position.reset_index())[:10]

In [11]:
test

,index,prop_lat,prop_long,school_lat,school_long
0,0,-37.780782,144.813020,-37.78219,144.82020
1,1,-37.869028,144.807250,-37.86390,144.81167
2,2,-37.276411,142.920079,-37.27736,142.92100
3,3,-37.292905,142.921881,-37.28600,142.92196
4,4,-37.782361,144.808833,-37.78753,144.80752
5,5,-37.860329,145.022016,-37.85807,145.03116
6,6,-37.773066,144.930304,-37.77432,144.92612
7,7,-37.757388,144.869104,-37.75847,144.86162
8,8,-37.557037,143.879475,-37.56170,143.87401
9,9,-36.556619,145.973497,-36.55396,145.97412


In [30]:
positions = []
for i in range(len(test)-1):
    cur_position = []
    cur_position.append([list(test[['prop_long']].iloc[i])[0], list(test[['prop_lat']].iloc[i])[0]])
    cur_position.append([list(test[['school_long']].iloc[i])[0], list(test[['school_lat']].iloc[i])[0]])
    positions.append(cur_position)

In [53]:
for each in positions:
    print(each)

[[144.8130199, -37.7807821], [144.8202, -37.78219]]
[[144.8072501, -37.8690276], [144.81167, -37.8639]]
[[142.920079, -37.27641089999999], [142.921, -37.27736]]
[[142.9218811, -37.292905], [142.92196, -37.286]]
[[144.8088328, -37.7823611], [144.80752, -37.78753]]
[[145.0220163, -37.860329], [145.03116, -37.85807]]
[[144.9303035, -37.7730663], [144.92612, -37.77432]]
[[144.8691038, -37.7573875], [144.86162, -37.75847]]
[[143.8794746, -37.5570369], [143.87401, -37.5617]]


In [61]:
routes = []
for each in positions:
    route = client.directions(
        coordinates=each
    )
    print(route)
    routes.append(route['routes'][0]['summary']['distance'])


{'routes': [{'summary': {'distance': 886.4, 'duration': 125.1}, 'segments': [{'distance': 886.4, 'duration': 125.1, 'steps': [{'distance': 360.4, 'duration': 58.9, 'type': 11, 'instruction': 'Head north on Selwyn Street', 'name': 'Selwyn Street', 'way_points': [0, 20]}, {'distance': 325.0, 'duration': 34.2, 'type': 0, 'instruction': 'Turn left onto King Edward Avenue', 'name': 'King Edward Avenue', 'way_points': [20, 32]}, {'distance': 130.5, 'duration': 15.1, 'type': 7, 'instruction': 'Enter the roundabout and take the 3rd exit onto Adelaide Street', 'name': 'Adelaide Street', 'exit_number': 3, 'way_points': [32, 48]}, {'distance': 70.5, 'duration': 16.9, 'type': 0, 'instruction': 'Turn left onto Drummartin Street', 'name': 'Drummartin Street', 'way_points': [48, 50]}, {'distance': 0.0, 'duration': 0.0, 'type': 10, 'instruction': 'Arrive at Drummartin Street, on the left', 'name': '-', 'way_points': [50, 50]}]}], 'bbox': [144.813233, -37.78264, 144.820127, -37.77998], 'geometry': '`ar

{'routes': [{'summary': {'distance': 1886.1, 'duration': 227.9}, 'segments': [{'distance': 1886.1, 'duration': 227.9, 'steps': [{'distance': 22.4, 'duration': 5.4, 'type': 11, 'instruction': 'Head south on The Terrace', 'name': 'The Terrace', 'way_points': [0, 1]}, {'distance': 11.2, 'duration': 2.7, 'type': 6, 'instruction': 'Continue straight onto The Terrace', 'name': 'The Terrace', 'way_points': [1, 2]}, {'distance': 208.4, 'duration': 25.0, 'type': 1, 'instruction': 'Turn right onto St Georges Road', 'name': 'St Georges Road', 'way_points': [2, 3]}, {'distance': 543.5, 'duration': 46.2, 'type': 1, 'instruction': 'Turn right onto Kooyong Road', 'name': 'Kooyong Road', 'way_points': [3, 17]}, {'distance': 764.5, 'duration': 81.2, 'type': 1, 'instruction': 'Turn right onto High Street, 24', 'name': 'High Street, 24', 'way_points': [17, 30]}, {'distance': 153.6, 'duration': 23.6, 'type': 1, 'instruction': 'Turn right onto Glenferrie Road, 19', 'name': 'Glenferrie Road, 19', 'way_point

In [62]:
routes

[886.4, 1070.1, 374.5, 1182.0, 3872.8, 1886.1, 568.2, 2078.8, 955.5]

In [58]:
route

{'type': 'FeatureCollection',
 'features': [{'bbox': [143.873966, -37.561854, 143.879507, -37.557216],
   'type': 'Feature',
   'properties': {'segments': [{'distance': 1075.2,
      'duration': 774.2,
      'steps': [{'distance': 82.3,
        'duration': 59.3,
        'type': 11,
        'instruction': 'Head west on Johns Street',
        'name': 'Johns Street',
        'way_points': [0, 1]},
       {'distance': 66.6,
        'duration': 48.0,
        'type': 0,
        'instruction': 'Turn left onto Oliver Street',
        'name': 'Oliver Street',
        'way_points': [1, 2]},
       {'distance': 223.3,
        'duration': 160.8,
        'type': 1,
        'instruction': 'Turn right onto Scott Parade',
        'name': 'Scott Parade',
        'way_points': [2, 5]},
       {'distance': 247.5,
        'duration': 178.2,
        'type': 0,
        'instruction': 'Turn left onto Queen Street North',
        'name': 'Queen Street North',
        'way_points': [5, 14]},
       {'distance'